<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>Linear Binary Classification by Linear Programming</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.

**NOTE:** Run the following script whenever running this script on a Google Colab.

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

# 4. Linear Classification by Integer Linear Programming
In this lab session, you have to experiment with **(Integer) Linear Programming** to train a **binary linear classifier** for solving three different problems:

1. The classification two classes of random points drawn from two different 2D Gaussian distributions.
2. The classification of euro banknotes as *regular* or *fake*, given 4 attributes of banknote images:
    * Variance of Wavelet Transformed image (continuous)
    * Skewness of Wavelet Transformed image (continuous)
    * Curtosis of Wavelet Transformed image (continuous)
    * Entropy of image (continuous)
3. The classification of breast cancer as either malignant (M) or benign (B), given 30 attributes referring to 10 features described below.

Your model will be evaluated in terms of overall accuracy, given as the percentage of object classified correctly as either positive or negative.

For the banknote and breast cancer classification problems, you have only a subset of the whole dataset. The missing data will be used to nominate the best classifier proposed by the different student groups.

To design your classifier, you can only use **(Integer) Linear Programming**. If you will try after the lecture other approaches studied in other courses (e.g., SVM, decision tress, or Neural Networks), I will be very curious the hear about.

**REMARK:** If working on your personal computer, you might want to install [Gurobi with an academic license](https://www.gurobi.com/academia/academic-program-and-licenses/). Gurobi is a commercial solver that is extremely fast in solving large ILP instances, much more faster than the free GLPK solver.

## 4.1 Gaussian samples in 2D
The first dataset is generated randomly with the following code, which generates a list of $3n$ points in the plane. The first $2n$ points have both coordinates with mean equal to 2.0 and standard deviations $d$ (=0.5 by default), and they belong to the first class; the remaining $n$ points have coordinates with mean equal to 4.0 and standard deviations $d$, and they belong to the second class.

In [ ]:
import numpy as np

def Gaussian(n, mu, sigma):
    return np.random.normal(mu, sigma, (n, 2))

def RandomData(n, d=0.5):
    # To experiment with different random dataset, comment the following line
    np.random.seed(17)
    
    # Generate points
    As = Gaussian(2*n, 2, d)
    Bs = Gaussian(n, 4, d)       
    Xs = []
    Ys = []    
    for a in As:
        Xs.append(a)
        Ys.append(0) # First class
    for a in Bs:
        Xs.append(a)
        Ys.append(1) # Second class
        
    return Xs, Ys

Xs, Ys = RandomData(50, 0.25)

print(Xs[0], Ys[0])

In practice, you have generated $m$ points $x_i$ with labels $y_i$, such that the points with $y_i=0$ belongs to the first class, and the point $y_i=1$ belongs to the second class.

You have to find a Linear Classifier in 2D, that is, you have to find the hyperplane defined by the vector $(a_0, a_1, a_2)$, such that:

$$
a_0 x_{i0} + a_1 x_{i1} + a_2 \geq 0 \quad \text{if $x_i$ belong to the first class } (y_i=0)\\
a_0 x_{i0} + a_1 x_{i1} + a_2 < 0 \quad \text{if $x_i$ belong to the second class } (y_i=1)\\
$$

For instance, the (very bad!) linear classifier specified by $(a_0, a_1, a_2)=(1,-1,0.5)$, classifies each point of the dataset as shown in the following graphical representation.

In [ ]:
import matplotlib.pyplot as plt

def PlotSolution(Xs, Ys, A):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    # Left plot
    ax1.scatter([x[0] for x in Xs], [x[1] for x in Xs], 
                color=['green' if y == 1 else 'blue' for y in Ys],
                alpha=0.35)
    # Right plot
    ax2.scatter([x[0] for x in Xs], [x[1] for x in Xs], 
                color=['green' if y == 1 else 'blue' for y in Ys],
                alpha=0.35)
    
    xmin = min(x[0] for x in Xs)
    xmax = max(x[1] for x in Xs)
    x = np.linspace(xmin, xmax, 10)
    
    # Draw the classifier line (over the plot domain)
    y = -A[0]/A[1]*x + A[2]/A[1]
    
    ax2.plot(x, y, color='red')
    
    # Miss-classifications
    Vs = []
    for i,x in enumerate(Xs):
        if A[0]*x[0] + A[1]*x[1] < A[2] and Ys[i] == 0:
            Vs.append(x)                        
        else:
            if A[0]*x[0] + A[1]*x[1] > A[2] and Ys[i] == 1:
                Vs.append(x)
    
    ax2.scatter([x[0] for x in Vs], [x[1] for x in Vs], color='red', alpha=0.5, marker='x')
    
    # Final plot
    ax1.axis([xmin-0.5, xmax+0.5, 1, 5.5])
    ax2.axis([xmin-0.5, xmax+0.5, 1, 5.5])
    ax1.axis('equal')
    ax2.axis('equal')
    # plt.savefig('lin_classifier.pdf') 
    plt.show() 
    
# HOW-TO PLOT
PlotSolution(Xs, Ys, [-1,-1,-6])  # <<<<============

We can evaluate the classifier in terms of *accuracy* and/or by using the *confusion matrix*.

In [ ]:
def Accuracy(A, Bx, Bl):
    # Count overall miss-classifications
    v = 0
    for xs, y in zip(Bx, Bl):
        ax = sum(x*a for x,a in zip(xs, A[:-1]))
        if ax <= A[-1] and y == 0:
            v += 1
        if ax > A[-1] and y == 1:
            v += 1    
    return round((len(Bx)-v)/len(Bx)*100, 3), len(Bx)-v, len(Bx)

def Confusion(A, Bx, Bl):
    # Compute in order:
    # True Positive, False Positive, True Negative, False Negative
    tp, fp, tn, fn = 0, 0, 0, 0    
    for xs, y in zip(Bx, Bl):
        ax = sum(x*a for x,a in zip(xs, A[:-1]))
        if ax >= A[-1] and y == 0:
            tn += 1
        if ax < A[-1] and y == 1:
            tp += 1
            
        if ax < A[-1] and y == 0:
            fn += 1
        if ax >= A[-1] and y == 1:
            fp += 1    
    return tp, fp, tn, fn

# HOW-TO EVALUATE  # <<<<============
print("Accuracy:", Accuracy([1,-1,0.5], Xs, Ys))
print("Confusion Matrix:", Confusion([1,-1,0.5], Xs, Ys))

**Note:** This classifier is indeed very poor: flipping a coin could give better results.

**EXERCISE 1:** Design your best possible Linear Classifier using Integer Linear Programming and Pyomo. Use the visual representation in 2D to get intuition on your classifier. Try to change the initial random distributions (mean and deviation).

Implement your solution in the following function:

In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory
from pyomo.environ import Binary, RangeSet, NonNegativeReals

def LinearClassifier(Xs, Ys):
    # TO COMPLETE WITH YOUR MODEL
    return [1, -1, 0.5]


# HOW-TO TEST YOUR SOLUTION
A = LinearClassifier(Xs,Ys)  # <<<<============

print("Accuracy:", Accuracy(A, Xs, Ys))
print("Confusion Matrix:", Confusion(A, Xs, Ys))
PlotSolution(Xs, Ys, [1,-1,0.5])

## 4.2 Banknote fake classification
In this second exercise, you have to design a Linear Classifier to distinguish between original and fake banknotes.

Each single banknote is first digitalized, and second, 4 features of each image are reported in the dataset. The 4 features that you can use are:

* Variance of Wavelet Transformed image (continuous)
* Skewness of Wavelet Transformed image (continuous)
* Curtosis of Wavelet Transformed image (continuous)
* Entropy of image (continuous)

Your are given a subset of 992 banknotes that you can use to train your model.

**GROUP CHALLENGE 1:** Design your best Linear Classifier which will achieve the best **accuracy** on a subset of 380 banknotes that you cannot have access to. 

**DATA:** The data about the banknotes is given via a csv file, with a row for each banknote. The first 4 fields of each row gives the features of the banknote, the last field, while gives a binary label: 0 or 1. You can parse the data with the following snippet.

In [ ]:
# Run this command to import the dataset
!wget http://www-dimat.unipv.it/gualandi/opt4ds/banknote_train.csv

In [ ]:
# Parse the training set
def ParseData(filename):
    fh = open(filename, 'r', encoding="utf-8")
    Xs = []
    Ys = []
    for line in fh:
        row = line.replace('\n','').split(',')        
        Xs.append( list(map(float, row[:-1])) )
        Ys.append( int(row[-1]) )
    return Xs, Ys  

# HOW-TO PARSE
Xs, Ys = ParseData('banknote_train.csv')
for i in range(5):
    print(Xs[i], Ys[i])

**Note:** You may want to split your data into training and test subsets, in order to verify how your model generalize to *unseen* data (but you are not forced to). In the later case, you can use the following code:

In [ ]:
def SplitTrainTestSet(Xs, Ys, t=0.3):
    Xtrain, Ytrain = [], []  # Train sets
    Xtest,  Ytest = [], []  # Test sets
    
    np.random.seed(13)
    
    for x, y in zip(Xs, Ys):
        if np.random.uniform(0, 1) > t:
            Xtrain.append(x)
            Ytrain.append(y)
        else:
            Xtest.append(x)
            Ytest.append(y)
            
    return Xtrain, Ytrain, Xtest, Ytest

# HOW-TO USE A DATA SPLITTING
Xtrain, Ytrain, Xtest, Ytest = SplitTrainTestSet(Xs, Ys)  

In [ ]:
def LinearClassifierBank(Xs, Ys):
    # TO COMPLETE WITH YOUR DATA
    return []

## 4.3 Breast Cancer Classification
In this third exercise, you have to design a Linear Classifier to distinguish between malignant and benign breast cancer tumors, using a set of 30 features precomputed from digitized images of a fine needle aspirate (FNA) of a breast mass.

Each single breast mass is first analyzed, and then, the 30 features of each image are reported in the csv file `breast_cancer_train.csv`.

Attribute Information, in order by columns as appearing in the CSV file:

1. ID number
2. Diagnosis (M = malignant, B = benign)
3. radius (mean of distances from center to points on the perimeter)
4. texture (standard deviation of gray-scale values)
5. perimeter
6. area
7. smoothness (local variation in radius lengths)
8. compactness (perimeter^2 / area - 1.0)
9. concavity (severity of concave portions of the contour)
10. concave points (number of concave portions of the contour)
11. symmetry
12. fractal dimension ("coastline approximation" - 1)

For the features from column 3 to 12, the mean is reported with 4 significant digits.
The following 10 columns (from 13 to 22) report the standard error of the corresponding feature.
The remaining 10 columns (from 23 to 32) report the largest number (worst) of the corresponding feature.

For instance, column 3 is Mean Radius, column 13 is Radius SE, and column 23 is Worst Radius.

All feature values are recoded with four significant digits.

Follow the following steps to import the dataset and then work on your classifier.

In [ ]:
# Run this command to import the dataset
!wget http://www-dimat.unipv.it/gualandi/opt4ds/breast_cancer_train.csv

To parse the data, you can use the following basic parser.

In [ ]:
def ParseCancer(filename):
    fh = open(filename, 'r', encoding="utf-8")
    Xs = []
    Ys = []
    for line in fh:
        row = line.replace('\n','').split(',')        
        Xs.append( list(map(float, row[2:])) )
        Ys.append( int(row[1] == 'M') )
    return Xs, Ys    

Then, you can work out your solution.

In [ ]:
def LinearClassifierBreastCancer(Xs, Ys):
    # TO COMPLETE WITH YOUR DATA
    return []

**GROUP CHALLENGE 2:** Design your best Linear Classifier which will achieve the best **accuracy** on a subset of samples that you do not have access to. 

## 4.4 Submission and Evaluation
Each group (with 1 or 2 students per group) must submit a solution by email before Thursday, 23rd, at midnight.

All the submitted classifier will be evaluated during the lecture on Friday, 24th, by using an **unseen** validation dataset.

The best group on the unseen dataset will get two extra "bonus" points to the final oral exam. All the group submitting a working solution will get an extre point to the final oral exam.
